# DistilBERT Model Evaluation with Another Dataset

## 1. Load the saved model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_DIR = "./saved_db/distilbert_merged_full_final" 
eval_tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
eval_model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
eval_model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## 2. Load the pre-processed dataset

In [2]:
from datasets import load_dataset

TEST_CSV = "./processed_df3_60k_fe.csv"

# Load test-only split from the CSV
raw = load_dataset("csv", data_files={"test": TEST_CSV})["test"]

# Figure out the label column name (common possibilities)
possible_label_cols = ["generated", "label", "labels", "target", "y"]
label_col = next((c for c in possible_label_cols if c in raw.column_names), None)
if label_col is None:
    raise ValueError(f"Could not find a label column in {raw.column_names}. "
                     "Please rename your ground-truth column to one of: "
                     f"{possible_label_cols}")

# Make sure there's a 'text' column
if "text" not in raw.column_names:
    raise ValueError(f"Could not find 'text' column. Found: {raw.column_names}. "
                     "Please ensure the CSV has a 'text' column.")

def tok_fn(batch):
    return eval_tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

test_ds = raw.map(tok_fn, batched=True)
test_ds = test_ds.rename_column(label_col, "labels")

# Keep only the needed columns for PyTorch
keep_cols = ["input_ids", "attention_mask", "labels"]
test_ds = test_ds.remove_columns([c for c in test_ds.column_names if c not in keep_cols])
test_ds.set_format("torch", columns=keep_cols)

len(test_ds), test_ds[0]


Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/59342 [00:00<?, ? examples/s]

(59342,
 {'labels': tensor(0),
  'input_ids': tensor([  101,  2007,  3008,  2119,  6800,  2013,  3836,  6667,  1010,  4252,
           2467,  2038,  2019,  9788, 11084,  2000,  2033,  1012,  1045,  2318,
           2026, 14924,  2075,  2449,  1999,  3694,  2184,  2005,  2070,  4979,
           2769,  1010,  2021, 14924,  2075,  2003,  2172,  2062, 10368,  2084,
           1045,  8078,  1012,  1045,  2147,  2007,  2493,  2040,  9652,  2734,
           2000,  3413,  2037,  5352,  2664,  2018,  2069,  2048,  3134,  2187,
           1999,  2082,  1012,  7297,  1010,  1045,  2031,  4036,  2493,  2007,
           1040,  7274,  2571, 14787,  2021,  2145,  2359,  2000,  7323,  3020,
           2495,  1012,  2026,  6587,  3076,  2003,  2069,  1019,  2021,  4587,
           2003,  1999,  2010,  2753,  2015,  1012,  2011,  2551,  2007,  2111,
           2013,  2367,  3834, 15406,  1998,  5535,  1010,  1045,  2428,  2288,
           2000,  2156,  1996,  2088,  1998,  4553,  2129,  2000, 10639,  64

## 3. Evaluate

In [3]:
import numpy as np
from transformers import Trainer
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support
)

# Use a bare Trainer for prediction convenience
eval_trainer = Trainer(model=eval_model, tokenizer=eval_tokenizer)

pred_output = eval_trainer.predict(test_ds)
logits = pred_output.predictions
y_pred = np.argmax(logits, axis=1)
y_true = pred_output.label_ids

# Overall scores
acc = accuracy_score(y_true, y_pred)
prec_weighted, rec_weighted, f1_weighted, _ = precision_recall_fscore_support(
    y_true, y_pred, average="weighted", zero_division=0
)
prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="macro", zero_division=0
)
prec_micro, rec_micro, f1_micro, _ = precision_recall_fscore_support(
    y_true, y_pred, average="micro", zero_division=0
)

print("=== Aggregate Metrics ===")
print(f"Accuracy          : {acc:.4f}")
print(f"Precision (macro) : {prec_macro:.4f}")
print(f"Recall (macro)    : {rec_macro:.4f}")
print(f"F1 (macro)        : {f1_macro:.4f}")
print(f"Precision (micro) : {prec_micro:.4f}")
print(f"Recall (micro)    : {rec_micro:.4f}")
print(f"F1 (micro)        : {f1_micro:.4f}")
print(f"Precision (weighted): {prec_weighted:.4f}")
print(f"Recall (weighted)   : {rec_weighted:.4f}")
print(f"F1 (weighted)       : {f1_weighted:.4f}")

# Per-class report (includes precision/recall/F1 and support by class)
print("\n=== Classification Report (per class) ===")
print(classification_report(y_true, y_pred, digits=4, zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("=== Confusion Matrix ===")
print(cm)


C:\Users\Dayso\AppData\Local\Temp\ipykernel_44860\3027606125.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(model=eval_model, tokenizer=eval_tokenizer)


=== Aggregate Metrics ===
Accuracy          : 0.6164
Precision (macro) : 0.6730
Recall (macro)    : 0.6163
F1 (macro)        : 0.5821
Precision (micro) : 0.6164
Recall (micro)    : 0.6164
F1 (micro)        : 0.6164
Precision (weighted): 0.6730
Recall (weighted)   : 0.6164
F1 (weighted)       : 0.5822

=== Classification Report (per class) ===
              precision    recall  f1-score   support

           0     0.7719    0.3301    0.4624     29660
           1     0.5741    0.9025    0.7018     29682

    accuracy                         0.6164     59342
   macro avg     0.6730    0.6163    0.5821     59342
weighted avg     0.6730    0.6164    0.5822     59342

=== Confusion Matrix ===
[[ 9790 19870]
 [ 2893 26789]]
